<h1>RadxRate Tutorial</h1>

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local PID category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuated-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

<h2>Overview</h2>


**1. Parameter files**
* RadxRate main params
* RadxRate Kdp_specific params
* RadxRate Pid_specific params
* RadxRate Rate_specific params
* Pid Thresholds params (Radar d
ependent)
* Grib2toMdv params

### **Parameter files modifications**
* **RadxRate main params**
    * input_dir (radar data) [line 105]
    * INPUT FIELD INFORMATION. [line 155]
        * SNR_available = TRUE; change to FALSE; [line 177]
        * DBZ_field_name = "DBZ"; change to "REF"; [line 215]
        * LDR_available = TRUE; change to FALSE [line 255]
    * kdp_params_file_path  [line 284]
    * pid_params_file_path  [line 303]
    * PID_use_attenuation_corrected_fields = TRUE [line 331]
    * RATE_params_file_path [line 350]
    * RATE_use_attenuation_corrected_fields = TRUE [line 378]
    * output_dir [line 699] -- or set in command line with -outdir flag
    * output_format = OUTPUT_FORMAT_CFRADIAL; (Default) [line 755] 
* **RadxRate Kdp_specific params**
    * KDP_fir_filter_len = KDP_FIR_LEN_10; [line 65]
    * KDP_psob_method = PEAK_REMOVAL_METHOD; [line 114]
* **RadxRate Pid_specific params**
    * PID_thresholds_file_path [line 27]
    * PID_use_soundings_from_spdb = FALSE; set to TRUE[line 303]
    * PID_sounding_spdb_url = "spdb/soundings"; [line 313]
    * PID_sounding_location_name [line 339]
    *  **do sounding steps first before filling Pid variables out**
* **RadxRate Rate_specific params**
    * N/A?
* **Pid Thresholds params**
    * Radar dependent
        * Nexrad is S-band simultaneous h/v transmission and recieving


https://stackoverflow.com/questions/19699059/print-directory-file-structure-with-icons-for-representation-in-markdown
```
BASE_DIR
│   RadxRate_tutorial.ipynb
│ 
│
└───data
│   │   ...
│   │   
│   │
│   │───raw
│   │   │   KMHX20180914_191822_V06.ar2v
│   │   │   gfsanl_4_2018091418.g2.tar
│   │   │   
│   │───CfRadial
│   │   │   ...
│   │   │
│   │   │
│   │───sounding 
│   │   │   ...
│   │   │
│    ───
│   
└───params
│   │   Pid_thresholds.sband.shv.txt
│   │   RadxRate_main_params
│   │   RadxRate_Kdp_params
│   │   RadxRate_Pid_params
│   │   RadxRate_rate_params



- Print out parameter files

In [1]:
# # create original parameter files
# !/usr/local/lrose/bin/RadxRate -print_params > ./params_ori/RadxRate_main_params
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > ./params_ori/RadxRate_Kdp_params
# !/usr/local/lrose/bin/RadxRate -print_params_pid > ./params_ori/RadxRate_Pid_params
# !/usr/local/lrose/bin/RadxRate -print_params_rate > ./params_ori/RadxRate_rate_params

# # setup parameter files for Users
# !/usr/local/lrose/bin/RadxRate -print_params > ./params/RadxRate_main_params
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > ./params/RadxRate_Kdp_params
# !/usr/local/lrose/bin/RadxRate -print_params_pid > ./params/RadxRate_Pid_params
# !/usr/local/lrose/bin/RadxRate -print_params_rate > ./params/RadxRate_rate_params
# !/usr/local/lrose/bin/Grib2toMdv -print_params > ./params/Grib2toMdv_params

In [2]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


Here we will set up the directory structure to simplify our commands. 

(**note: modify BASE_DIR and LROSE_DIR if your personal machine has different directory structures**)

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RADAR_NAME: the name of the radar used in this tutorial
* LROSE_DIR: path to the LROSE installation 

In [3]:
os.environ['BASE_DIR'] = '/home/jovyan/ams2023'
os.environ['RADAR_NAME'] = 'KMHX'
os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
!echo "Base directory: "$BASE_DIR
!echo "Base directory: "$RADAR_NAME

Base directory: /home/jovyan/ams2023
Base directory: KMHX


# Data download and directory organization

https://drive.google.com/drive/folders/1nQ1-CUR_OFuSdWgsE-qL35L5rcWTmEh4?usp=share_link

In [4]:
## make a directory for all the data files (raw and analysis)
!rm -rf ${BASE_DIR}/data/echo
!mkdir ${BASE_DIR}/data/echo

## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/echo/raw
!mkdir ${BASE_DIR}/data/echo/raw

## make subdirectory within data for sounding data
!rm -rf ${BASE_DIR}/data/echo/sounding
!mkdir ${BASE_DIR}/data/echo/sounding

### wget raw files from ftpserver and place in appropriate $BASE_DIR subdirectory
# !wget ftpserver/KMHX20180914_191822_V06.ar2v
# !wget ftpserver/gfsanl_4_2018091418.g2.tar

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO" -O gfsanl_4_2018091418.g2.tar && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10On5qmT4p-Wvm1kpdGjLUHdu0TQyXZDK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10On5qmT4p-Wvm1kpdGjLUHdu0TQyXZDK" -O KMHX20180914_191822_V06.ar2v && rm -rf /tmp/cookies.txt

## move files to proper directories
!mv KMHX20180914_191822_V06.ar2v ${BASE_DIR}/data/echo/raw
!mv gfsanl_4_2018091418.g2.tar ${BASE_DIR}/data/echo/raw


--2022-12-26 20:38:46--  https://docs.google.com/uc?export=download&confirm=t&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO
Resolving docs.google.com (docs.google.com)... 172.217.1.110, 2607:f8b0:4009:801::200e
Connecting to docs.google.com (docs.google.com)|172.217.1.110|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8u0hq8ai28ksjr64282060qg9p18ch61/1672087125000/06186746192597284684/*/1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO?e=download&uuid=d3b6a3d2-dcdb-46c4-9577-cb89ab7d3145 [following]
--2022-12-26 20:38:46--  https://doc-08-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8u0hq8ai28ksjr64282060qg9p18ch61/1672087125000/06186746192597284684/*/1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO?e=download&uuid=d3b6a3d2-dcdb-46c4-9577-cb89ab7d3145
Resolving doc-08-5g-docs.googleusercontent.com (doc-08-5g-docs.googleusercontent.com)... 142.250.190.65, 2607:f8b0:400

# Convert Nexrad Level II to CfRadial files

In [5]:
# !rm -rf ${BASE_DIR}/data/echo/CfRadial
# !mkdir ${BASE_DIR}/data/echo/CfRadial
!${LROSE_DIR}/RadxConvert -f ${BASE_DIR}/data/echo/raw/KMHX20180914_191822_V06.ar2v -outdir ${BASE_DIR}/data/echo/CfRadial


Program 'RadxConvert'
Run-time 2022/12/26 20:38:56.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to 

# Untar GFS analysis grib2 files for sounding
* Grb2 --> Mdv --> Spdb

In [6]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/echo/sounding/grib
!mkdir ${BASE_DIR}/data/echo/sounding/grib

# extract file from tape archive (tar)
!tar -xvf ${BASE_DIR}/data/echo/raw/gfsanl_4_2018091418.g2.tar -C ${BASE_DIR}/data/echo/sounding/grib/


gfs_4_20180914_1800_000.grb2
gfs_4_20180914_1800_003.grb2
gfs_4_20180914_1800_006.grb2


- **Grib2toMdv**
    - input_dir [line 69] optional, if directory is specified in command line
    - write_non_forecast [line 385]
    - non_forecast_mdv_url (location of outputted mdv files) [line 395]
    - out_projection_info [line 715] Radar location 
    - out_grid_info [line 749] Subjective box around radar(s)
    - height_levels [line 838] Height levels, optional

In [7]:
# convert model analysis to Mdv
!${LROSE_DIR}/Grib2toMdv -params ${BASE_DIR}/params/echo/Grib2toMdv_params -f ${BASE_DIR}/data/echo/sounding/grib/*.grb2


Program 'Grib2toMdv'
Run-time 2022/12/26 20:39:07.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to c

- **Mdv2SoundingSpdb**
    - input data field names [line 126] 
    - sampling locations [line 264]
        - Need to input location of current radar KMHX
    - ouput_url [line 363]

In [8]:
# convert Mdv to SoundingSpdb
!${LROSE_DIR}/Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180914/*.mdv*
!${LROSE_DIR}/Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180915/*.mdv*


Program 'Mdv2SoundingSpdb'
Run-time 2022/12/26 20:39:15.

Copyright (c) 1992 - 2022
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as no

- Check sounding data is filled (except vertical velocity and divergence)

In [9]:
!${LROSE_DIR}/SpdbQuery -url ${BASE_DIR}/data/echo/sounding/spdb -start "2018 09 14 00 00 00" -end " 2018 09 14 23 00 00"


url: /home/jovyan/ams2023/data/echo/sounding/spdb
Prod label: Sounding Data
Prod id:    301
DataType: 0
N Chunks:   2

Header for chunk ---> 0 <---
  data_type: 1481133387 (KMHX)
  data_type2: 0
  valid_time: 2018/09/14 18:00:00
  expire_time: 2018/09/15 06:00:00
  write_time: 2022/12/26 20:39:15
  len: 1448
  current_compression: none
  stored_compression: none


Data for sounding:
Sounding Product
  launch time      = 2018/09/14 18:00:00
  number of points = 29
  sourceId         = 3
  leadSecs         = 0
  spare int[0] = 0
  spare int[1] = 0
  lat = 34.775902
  lon = -76.876198
  alt = 43.000000
  missing = -9999.000000
  spare float[0] = 0.000000
  spare float[1] = 0.000000
  sourceName = RUC
  sourceFmt  = Mdv2SoundingSpdb
  siteName   = KMHX
  Sounding Point 0
    pressure = 1000.000000
    altitude = 0.000000
    u  wind  = -15.068794
    v  wind  = 9.757457
    w  wind  = -9999.000000
    relHum   = 84.960449
    temp     = 25.810978
    div      = -9999.000000
    spare[0] = 

# Setup steps complete, run RadxRate

In [ ]:
!${LROSE_DIR}/RadxRate -params ${BASE_DIR}/params/echo/RadxRate_main_params -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc -outdir ${BASE_DIR}/data/echo/rate


## Plot PID and rate results for NEXRAD Morehead City radar (KMHX)

In [ ]:
# Read CfRadial file into radar object
filePathRate = "/home/jovyan/ams2023/data/echo/rate/20180914/cfrad.20180914_191822.077_to_20180914_192511.621_KMHX_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(filePathRate)
rate_kmhx.info('compact')


In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0]],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 13.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(250.)
displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

pid_cbar = displayRate.cbs[3]
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
# pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
# pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()